Script for downloading likes:

Endptt 1 - Download most recent 100 liking users given a tweet id
    - 75 requests per 15-minute
Endpt 2 - Download a user's liked tweets. 
    - let's restrict this to most recent 1000 likes
    - 75 requests per 15-minute
    - goes towards tweet cap of 500,000 per month
    
    
Approach:
1) For each labelled account, GET most recent 50 tweets that are not retweets or replies
    - Client.get_users_tweets(twitter_id, exclude=['retweets', 'replies'], max_results=50]
2) For each tweet returned, get users who liked it
    - Client.get_liking_users(tweet_id, 
    
    

Only include accounts with <1000 follows

Mentions:
1) Gets tweets that mention the user. Retrieve most recent 50 tweets that mention user
- Client.get_users_mentions(twitter_id, max_results=100, pagination_token=next_page_token)
- 180 requests per 15-minute

In [430]:
import pandas as pd
import numpy as np
import networkx as nx
import json
import requests

In [431]:
import tweepy
from tweepy import Client

key = ""
secret = ""
token = ""

api = tweepy.Client(bearer_token=token, consumer_key=key, consumer_secret=secret, 
              return_type=requests.Response,
              wait_on_rate_limit=True)

In [432]:
import os

follows_processed = []
mentions_already_processed = []
 
for name in os.listdir('bot'):
    if os.path.isdir(name):
        continue
    list_name = name.replace('.pkl','').replace('edgelist_2_edges_from_start_','')
    follows_processed.append(int(list_name))
    
    
for name in os.listdir('mentions/bot'):
    if os.path.isdir(name):
        continue
    list_name = name.replace('.pkl','').replace('edgelist_mentions_','')
    mentions_already_processed.append(int(list_name))

In [433]:
len(os.listdir('mentions/human'))

265

In [434]:
len(os.listdir('mentions/bot'))

194

In [274]:
follows_processed

[1005564252948582401,
 1030869656666599424,
 1016773517587537920,
 1661404440,
 4307623643,
 747958680386179072,
 1416121214,
 821215184660393986,
 729462247647551488,
 2855732920,
 844779426260791296,
 913028718377005056,
 352132252,
 837316817324556289,
 875984489138200577,
 1004764640486854656,
 1058130850578137089,
 2928556470,
 2901953015,
 2385417602,
 790017240733278208,
 490645569,
 4459687272,
 2491508162]

In [275]:
mentions_already_processed

[1016773517587537920,
 1030869656666599424,
 4459687272,
 875984489138200577,
 913028718377005056,
 1005564252948582401,
 844779426260791296,
 837316817324556289,
 729462247647551488,
 2928556470,
 821215184660393986,
 747958680386179072,
 2385417602,
 2901953015,
 1004764640486854656,
 1058130850578137089,
 352132252,
 790017240733278208,
 1661404440,
 1416121214,
 4307623643]

In [435]:
combined = pd.read_pickle('node_and_degree_list.pkl')
small = combined.loc[combined.followers_count < thresh]
small = small.loc[small.followers_count < thresh]


In [436]:
small.label.value_counts()

human    265
bot      193
Name: label, dtype: int64

In [276]:
# Get neighbors of a node
def add_this_nodes_edgelist(this_acc, curr_edgelist):
    visited.add(this_acc)

    follow_list = []    # list of user id's who show up in first 100 tweets mentioning this user

    # Get the users who mentioned this user and add to edgelist
    request = api.get_users_mentions(this_acc, max_results=100, 
                                     expansions='author_id', 
                                     user_fields=['public_metrics'])
    data = request.json()

    # for each mentioning the originating account, record account id, 
    try:
        for acc in data['data']:
            follow_list.append(acc['author_id'])

    # Catch errors if not authorized to view user
    except Exception as e:
        print('Exception: issue retrieving follow lists for this account -- Skipping.')
        return curr_edgelist
        
    # aggregate # mentions for user as edge weights
    weighted_follow_list = [[x,follow_list.count(x)] for x in set(follow_list)]
    edges_to_add = pd.DataFrame(weighted_follow_list).rename(columns={
                                                                0: 'originating_id',
                                                                1: 'edge_weight'
                                                            })
    edges_to_add['receiving_id'] = this_acc
    curr_edgelist = pd.concat([curr_edgelist, edges_to_add])

    return curr_edgelist

In [277]:
def transition_to_next_separation_degree(curr_edgelist, degrees_of_separation):
    try:
       # create list of all neighbors in curr_edgelist df
        unvisited_neighbors = list(curr_edgelist.originating_id.append(curr_edgelist.receiving_id).unique())

        # if any nodes are in visited set, remove
        [unvisited_neighbors.remove(i) for i in unvisited_neighbors if i in visited]

        print()
        print(f'Current Number of edges from starting node {degrees_of_separation}')
        print(f'Num unvisited neighbors: {len(unvisited_neighbors)}')
        print('------')
        degrees_of_separation +=1 
    
        return curr_edgelist, degrees_of_separation, unvisited_neighbors
    
    
    except Exception as e:
        print(e)
        print('--------------------EXCEPTION--------------------')
        return curr_edgelist, degrees_of_separation, []

In [278]:
from tqdm.notebook import tqdm

In [419]:
mentions_already_processed# = []

[837316817324556289,
 913028718377005056,
 790017240733278208,
 1016773517587537920,
 2855732920,
 1416121214,
 821215184660393986,
 1004764640486854656,
 352132252,
 1661404440,
 2928556470,
 2901953015,
 2491508162,
 490645569,
 300415996,
 709807650,
 976973737,
 2385417602,
 4808016715,
 1433472060,
 449470627,
 1421859620,
 1334673584,
 1524730152,
 875984489138200577,
 1567185626,
 972532340,
 170568448,
 139954867,
 225313169,
 185218294,
 2861336274,
 288773752,
 3166813130,
 419353120,
 263300013,
 211001312,
 3018024769,
 6781802,
 37260832,
 179201716,
 2309880170,
 377341555,
 205990455,
 1036461709,
 4307623643,
 844779426260791296,
 4113621027,
 85989198,
 3082647453,
 2327994013,
 2221432382,
 28403745,
 705187969,
 1081628124,
 2943566101,
 105821431,
 4459687272,
 3098765088,
 176769813,
 2955302197,
 1494549962,
 608171108,
 728249330,
 87307603,
 3990931,
 1201545410,
 16257347,
 1098366914,
 3137992297,
 2334005046,
 223617437,
 36165609,
 587364967,
 2416820011,
 3

In [439]:
add_this_nodes_edgelist(starting_acc, edgelist)

,originating_id,receiving_id,edge_weight
0,49484837,3241838887,1
1,108086497,3241838887,1
2,54395146,3241838887,1
3,175443281,3241838887,1
4,32318320,3241838887,4
...,...,...,...
61,813349266093699073,3241838887,1
62,25476821,3241838887,1
63,899120112,3241838887,1
64,4601166861,3241838887,1


In [438]:
# Create Edgelists for each account in the filtered botometer dataset.
cols=['originating_id', 'receiving_id', 
          'edge_weight']
#mentions_already_processed = []
small_bot_accs = small.loc[small.label == 'bot']
label = 'bot'
for starting_acc in [3241838887, 1016773517587537920, 43310150][0:1]:#small_bot_accs.twitter_id:
#     if starting_acc in mentions_already_processed:
#         continue

    print(f'Starting node == {starting_acc}')
    # Initialization code for populating edgelist starting from node specified by "starting_acc"
    edgelist = pd.DataFrame(columns=cols)

    degrees_of_separation = 0

    visited = set()
    visited.add(starting_acc)

    edgelist = add_this_nodes_edgelist(starting_acc, edgelist) # contains neighbors of starting node
    edgelist, degrees_of_separation, unvisited_neighbors = transition_to_next_separation_degree(edgelist, degrees_of_separation)
    
    # iterate thru all unvisited neighbors with 2 degree of separation from starting (labeled) node
    # process df in chunks before concatenating
    while degrees_of_separation <= 1:
        curr_edgelist = pd.DataFrame(columns=cols)
        for i in tqdm(unvisited_neighbors):
            curr_edgelist = add_this_nodes_edgelist(i, curr_edgelist)
            visited.add(i)
        
        curr_edgelist, degrees_of_separation, unvisited_neighbors = transition_to_next_separation_degree(curr_edgelist, degrees_of_separation)
        edgelist = pd.concat([edgelist, curr_edgelist])

    
    all_temp = pd.DataFrame(columns = ['originating_id', 'originating_follower_count',
       'originating_following_count'])
    all_temp2 = pd.DataFrame(columns = ['receiving_id', 'receiving_follower_count',
       'receiving_following_count'])

    # Add follower/following counts
    unique_ids = edgelist['originating_id'].append(edgelist['receiving_id']).unique()

    iterations = int(len(unique_ids)/100) + 1
    for i in range(iterations):
        left = i*100
        right = (i+1)*100

        temps = unique_ids[left:right]
        if len(temps) == 0:
            continue

        r = api.get_users(ids=list(temps), user_fields='public_metrics')

        followers = []
        following = []
        for acc in r.json()['data']:
            followers.append(acc['public_metrics']['followers_count'])
            following.append(acc['public_metrics']['following_count'])

        temp = pd.DataFrame([temps, followers, following]).transpose().rename(columns={
            0: 'originating_id',
            1: 'originating_follower_count',
            2:  'originating_following_count'
        })
        temp.originating_id = temp.originating_id.astype(str)

        temp2 = pd.DataFrame([temps, followers, following]).transpose().rename(columns={
            0: 'receiving_id',
            1: 'receiving_follower_count',
            2:  'receiving_following_count'
        })
        temp2.receiving_id = temp2.receiving_id.astype(str)

        
        all_temp = pd.concat([all_temp, temp])
        all_temp2 = pd.concat([all_temp2, temp2])

        with_counts = edgelist.merge(all_temp, on='originating_id', how='left')
        with_counts = with_counts.merge(all_temp2, on='receiving_id', how='left')
        with_counts.drop_duplicates(inplace=True)
    
    # Save in directory
    fn = 'edgelist_mentions_' + str(starting_acc) + '.pkl'
    path = str('mentions/' + label) + '/' + fn
    with_counts.to_pickle(path)
    
    # Add twitter id to already processed list
    mentions_already_processed.append(starting_acc)

Starting node == 3241838887

Current Number of edges from starting node 0
Num unvisited neighbors: 66
------


  0%|          | 0/66 [00:00<?, ?it/s]

Exception: issue retrieving follow lists for this account -- Skipping.
Exception: issue retrieving follow lists for this account -- Skipping.

Current Number of edges from starting node 1
Num unvisited neighbors: 2289
------


In [421]:
len(mentions_already_processed)

458

In [386]:
all_temp = pd.DataFrame(columns = ['originating_id', 'originating_follower_count',
       'originating_following_count'])
all_temp2 = pd.DataFrame(columns = ['receiving_id', 'receiving_follower_count',
       'receiving_following_count'])

In [396]:
# Add follower/following counts
unique_ids = edgelist['originating_id'].append(edgelist['receiving_id']).unique()

iterations = int(len(unique_ids)/100) + 1
for i in range(iterations):
    left = i*100
    right = (i+1)*100

    temps = unique_ids[left:right]
    if len(temps) == 0:
        print('ooip')

    r = api.get_users(ids=list(temps), user_fields='public_metrics')

    followers = []
    following = []
    for acc in r.json()['data']:
        followers.append(acc['public_metrics']['followers_count'])
        following.append(acc['public_metrics']['following_count'])

    temp = pd.DataFrame([temps, followers, following]).transpose().rename(columns={
        0: 'originating_id',
        1: 'originating_follower_count',
        2:  'originating_following_count'
    })
    temp.originating_id = temp.originating_id.astype(str)

    temp2 = pd.DataFrame([temps, followers, following]).transpose().rename(columns={
        0: 'receiving_id',
        1: 'receiving_follower_count',
        2:  'receiving_following_count'
    })
    temp2.receiving_id = temp2.receiving_id.astype(str)

    all_temp = pd.concat([all_temp, temp])
    all_temp2 = pd.concat([all_temp2, temp2])

with_counts = edgelist.merge(all_temp, on='originating_id', how='left')
with_counts = with_counts.merge(all_temp2, on='receiving_id', how='left')


# Save in directory
fn = 'edgelist_mentions_' + str(starting_acc) + '.pkl'
path = str('mentions/' + label) + '/' + fn
with_counts.to_pickle(path)

# Add twitter id to already processed list
mentions_already_processed.append(starting_acc)

In [402]:
all_temp#.shape

,originating_id,originating_follower_count,originating_following_count
0,381831007,6050,8247
1,17137891,1276590,360
2,269091554,1378,892
3,55919541,3205,822
4,323439305,1259,1160
...,...,...,...
86,1034079672366067712,172,811
87,383401068,2528,701
88,726818840257171457,1099,1039
89,1062246222,233,164


In [398]:
all_temp2.shape

(891, 3)

In [409]:
with_counts = edgelist.merge(all_temp, on='originating_id', how='left')
with_counts = with_counts.merge(all_temp2, on='receiving_id', how='left')
